# 利用pysteps进行回波外推预报

## 第一步，下载雷达拼图数据

In [1]:
from nmc_met_io.retrieve_cmadaas import cmadaas_radarmosaic_cref_by_timerange
outpath = '/mnt/e/metradar_test/pysteps_data/mosaic_cref/'
cmadaas_radarmosaic_cref_by_timerange( time_range='[20250609100000,20250609120000]', 
                                     outpath=outpath)

File already exists: /mnt/e/metradar_test/pysteps_data/mosaic_cref//2025/06/09/RADA_L3_MST_CREF_QC/Z_RADA_C_BABJ_20250609100623_P_DOR_ACHN_CREF_20250609_100000.bin
File already exists: /mnt/e/metradar_test/pysteps_data/mosaic_cref//2025/06/09/RADA_L3_MST_CREF_QC/Z_RADA_C_BABJ_20250609101222_P_DOR_ACHN_CREF_20250609_100600.bin
File already exists: /mnt/e/metradar_test/pysteps_data/mosaic_cref//2025/06/09/RADA_L3_MST_CREF_QC/Z_RADA_C_BABJ_20250609101821_P_DOR_ACHN_CREF_20250609_101200.bin
File already exists: /mnt/e/metradar_test/pysteps_data/mosaic_cref//2025/06/09/RADA_L3_MST_CREF_QC/Z_RADA_C_BABJ_20250609102421_P_DOR_ACHN_CREF_20250609_101800.bin
File already exists: /mnt/e/metradar_test/pysteps_data/mosaic_cref//2025/06/09/RADA_L3_MST_CREF_QC/Z_RADA_C_BABJ_20250609103022_P_DOR_ACHN_CREF_20250609_102400.bin
File already exists: /mnt/e/metradar_test/pysteps_data/mosaic_cref//2025/06/09/RADA_L3_MST_CREF_QC/Z_RADA_C_BABJ_20250609103621_P_DOR_ACHN_CREF_20250609_103000.bin
File already exi

True

## 第二步，将二进制格式的数据转成pgm格式

In [2]:
%matplotlib inline
import matplotlib
matplotlib.use('Agg')  # 关键：禁用交互式后端
from metradar.io.pgmb_io import pgmb_write
import os
from metradar.io.read_new_mosaic_func import decode_mosaic
import xarray as xr
import numpy as np
import gzip


def zip_file(path, new_path):
    with open(new_path, 'wb') as wf:
        with open(path, 'rb') as rf:
            data = rf.read()
            # 压缩数据
            data_comp = gzip.compress(data,compresslevel=5)
        wf.write(data_comp)
 
 
filepath = '/mnt/e/metradar_test/pysteps_data/mosaic_cref/2025/06/09/RADA_L3_MST_CREF_QC/'

SLAT = 35
NLAT = 40
WLON = 115
ELON = 120.5

for filename in os.listdir(filepath):
    
    data = decode_mosaic(filepath,filename)
    # filename = 'ACHN.CREF000.20230421.180009.nc'
    obstimestr = filename.split('_')[9]+filename.split('_')[10][0:4].split('.')[0]
    outname = obstimestr + '_fmi.radar.composite.lowest_FIN_SUOMI1.pgm'
    outpath = '/mnt/e/metradar_test/pysteps_data/radar/fmi/%s'%filename.split('_')[9]
    if not os.path.exists(outpath):
        os.makedirs(outpath)

    file_name = outpath + os.sep + outname

    data = data.sel(lat=slice(SLAT, NLAT), lon=slice(WLON, ELON))

    width = data.CREF.shape[1]
    height = data.CREF.shape[0]
    maxval = 255
    gray = data.CREF*2+66

    aa = np.isnan(gray.data)
    gray.data[aa]=255
    cref = np.flipud(gray.data.astype(np.uint8))
    params={}
    params['obstimestr'] = obstimestr
    params['left_lon'] = WLON
    params['right_lon'] = ELON
    params['bottom_lat'] = SLAT
    params['upper_lat'] = NLAT
    pgmb_write ( file_name, params, width, height, maxval, cref )

    # gzip file

    # zip_file(file_name, file_name + '.gz')


    print('write pgmb file %s successfully!'%(file_name))


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

write pgmb file /mnt/e/metradar_test/pysteps_data/radar/fmi/20250609/202506091000_fmi.radar.composite.lowest_FIN_SUOMI1.pgm successfully!
write pgmb file /mnt/e/metradar_test/pysteps_data/radar/fmi/20250609/202506091006_fmi.radar.composite.lowest_FIN_SUOMI1.pgm successfully!
write pgmb file /mnt/e/metradar_test/pysteps_data/radar/fmi/20250609/202506091012_fmi.radar.composite.lowest_FIN_SUOMI1.pgm successfully!
write pgmb file /mnt/e/metradar_test/pysteps_data/radar/fmi/20250609/202506091018_fmi.radar.composite.lowest_FIN_SUOMI1.pgm successfully!
wr

## 第三步，调用pysteps的光流法进行外推预报

In [ ]:

from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
from pysteps import io, motion, nowcasts, rcparams, verification
from pysteps.utils import conversion, transformation
from pysteps.visualization import plot_precip_field, quiver
import os
import xarray as xr
from metradar.graph.draw_latlon_func import draw_latlon
from datetime import datetime, timedelta

###############################################################################
# Read the radar input images
# ---------------------------
#
# First, we will import the sequence of radar composites.
# You need the pysteps-data archive downloaded and the pystepsrc file
# configured with the data_source paths pointing to data folders.

# Selected case

date = datetime.strptime("202506091100", "%Y%m%d%H%M")
data_source = rcparams.data_sources["fmi"]
n_leadtimes = 12

###############################################################################
# Load the data from the archive
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

root_path = data_source["root_path"]
path_fmt = data_source["path_fmt"]
fn_pattern = data_source["fn_pattern"]
fn_ext = data_source["fn_ext"]
importer_name = data_source["importer"]
importer_kwargs = data_source["importer_kwargs"]
timestep = data_source["timestep"]

# Find the input files from the archive
fns = io.archive.find_by_date(
    date, root_path, path_fmt, fn_pattern, fn_ext, timestep, num_prev_files=2
)

# Read the radar composites
importer = io.get_method(importer_name, "importer")
Z, _, metadata = io.read_timeseries(fns, importer, **importer_kwargs)

# write Z to ds
# set longitude and latitude coordinates
slat = 35.5
nlat = 39.5
wlon = 115
elon = 120
lat = np.linspace(slat,nlat,Z.shape[1])
lon = np.linspace(wlon,elon,Z.shape[2])

# set time coordinates
curtime = metadata['timestamps'][-1]
curtime = np.array([curtime], dtype='datetime64[m]')
# data = np.expand_dims(data, axis=0)

# define coordinates
time_coord = ('time', curtime)
lon_coord = ('lon', lon, {
    'long_name':'longitude', 'units':'degrees_east', '_CoordinateAxisType':'Lon'})
lat_coord = ('lat', lat, {
    'long_name':'latitude', 'units':'degrees_north', '_CoordinateAxisType':'Lat'})

# create xarray
varattrs = {'long_name': 'Composite Refelectivity', 
            'short_name': 'cref', 'units': 'dBZ',
            'maxv':80,
            'minv':0}
data = xr.Dataset({'cref':(['lat', 'lon'], np.flipud(Z[-1]), varattrs)},
    coords={ 'lat':lat_coord, 'lon':lon_coord})

outpath = '/mnt/e/metradar_test/pysteps_nowcast/pic'

tstr = metadata['timestamps'][-1].strftime('%Y%m%d%H%M')
outname = '%s_obs.png'%tstr
dpi = 600
thred=10
draw_latlon(data.cref,data.lat.data,data.lon.data,slat,nlat,wlon,elon,outpath,outname,tstr,subtitle='实况',titlecolor='k',dpi=dpi,thred=thred)

# Convert to rain rate
# R, metadata = conversion.to_rainrate(Z, metadata)

# # Plot the rainfall field
# fig1 = plt.figure(figsize=(6, 6))
# plot_precip_field(Z[-1, :, :], geodata=metadata)
# # plt.show()

# Store the last frame for plotting it later later
# R_ = R[-1, :, :].copy()

# Log-transform the data to unit of dBR, set the threshold to 0.1 mm/h,
# set the fill value to -15 dBR
# R, metadata = transformation.dB_transform(R, metadata, threshold=0.1, zerovalue=-15.0)

# Nicely print the metadata
# pprint(metadata)

###############################################################################
# Compute the nowcast
# -------------------
#
# The extrapolation nowcast is based on the estimation of the motion field,
# which is here performed using a local tracking approach (Lucas-Kanade).
# The most recent radar rainfall field is then simply advected along this motion
# field in oder to produce an extrapolation forecast.

# Estimate the motion field with Lucas-Kanade
# st = time.time()
oflow_method = motion.get_method("LK")
V = oflow_method(Z[-3:, :, :])

# Extrapolate the last radar observation
extrapolate = nowcasts.get_method("extrapolation")
# R[~np.isfinite(R)] = metadata["zerovalue"]
Z_f = extrapolate(Z[-1, :, :], V, n_leadtimes)
for nn in range(len(Z_f)):
    fsttime = metadata['timestamps'][-1] + timedelta(minutes=10*(nn+1))
    tstr = fsttime.strftime('%Y%m%d%H%M')
    # set time coordinates
    fsttime = np.array([fsttime], dtype='datetime64[m]')
    # data = np.expand_dims(data, axis=0)

    # define coordinates
    time_coord = ('time', fsttime)
    lon_coord = ('lon', lon, {
        'long_name':'longitude', 'units':'degrees_east', '_CoordinateAxisType':'Lon'})
    lat_coord = ('lat', lat, {
        'long_name':'latitude', 'units':'degrees_north', '_CoordinateAxisType':'Lat'})

    # create xarray
    varattrs = {'long_name': 'Composite Refelectivity', 
                'short_name': 'cref', 'units': 'dBZ',
                'maxv':80,
                'minv':0}
    data = xr.Dataset({'cref':(['lat', 'lon'], np.flipud(Z_f[nn]), varattrs)},
        coords={ 'lat':lat_coord, 'lon':lon_coord})

    

    
    outname = '%s_fst.png'%tstr
    dpi = 600
    thred=10
    draw_latlon(data.cref,data.lat.data,data.lon.data,slat,nlat,wlon,elon,outpath,outname,tstr,subtitle='预报',titlecolor='r',dpi=dpi,thred=thred,add_title=1,prefix_title='雷达组合反射率拼图')    
   

pass
# Back-transform to rain rate
# R_f = transformation.dB_transform(R_f, threshold=-10.0, inverse=True)[0]

# et = time.time()
# print("Execution time(s): ", et - st)
# Plot the motion field
# fig2 = plt.figure(figsize=(6, 6))
# plot_precip_field(Z_f[-1,:,:], geodata=metadata)
# quiver(V, geodata=metadata, step=50)
# plt.show()

###############################################################################
# Verify with FSS
# ---------------
#
# The fractions skill score (FSS) provides an intuitive assessment of the
# dependency of skill on spatial scale and intensity, which makes it an ideal
# skill score for high-resolution precipitation forecasts.

# Find observations in the data archive
fns = io.archive.find_by_date(
    date,
    root_path,
    path_fmt,
    fn_pattern,
    fn_ext,
    timestep,
    num_prev_files=0,
    num_next_files=n_leadtimes,
)
# Read the radar composites
Z_o, _, metadata_o = io.read_timeseries(fns, importer, **importer_kwargs)
# R_o, metadata_o = conversion.to_rainrate(R_o, metadata_o, 223.0, 1.53)

# Compute fractions skill score (FSS) for all lead times, a set of scales and 1 mm/h
fss = verification.get_method("FSS")
scales = [2, 4, 8, 16, 32, 64, 128]
thr = 5.0
score = []
for i in range(n_leadtimes):
    score_ = []
    for scale in scales:
        score_.append(fss(Z_f[i, :, :], Z_o[i + 1, :, :], thr, scale))
    score.append(score_)



/mnt/e/metradar_test/pysteps_nowcast/pic/202506091100_obs.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091110_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091120_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091130_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091140_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091150_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091200_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091210_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091220_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091230_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091240_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091250_fst.png saved!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091300_fst.png saved!
file not found: /mnt/e/metradar_test/pysteps_data/radar/fmi/20250609/202506091212_fmi.radar.composite.l

## 将外推图片制作为gif动画

In [ ]:

from PIL import Image
import os


filepath = '/mnt/e/metradar_test/pysteps_data/pic'
outpath = filepath
filenames = os.listdir(filepath)
filenames = sorted(filenames)
images = []


lastfile=''
for filename in filenames:
    if filename.startswith('.') or filename.startswith('..'):
        continue
    if not filename.endswith('.png'):
        continue
    
    lastfile= filename
    images.append(Image.open(filepath + os.sep + filename))
    print(filename + ' added!')
outfile = outpath + os.sep + lastfile + '.gif'
images[0].save(outfile, format='GIF',
                    append_images=images[1:], save_all=True, duration=int(500), loop=0)
print(outfile + ' saved successfully!')


202506091100_obs.png added!
202506091110_fst.png added!
202506091120_fst.png added!
202506091130_fst.png added!
202506091140_fst.png added!
202506091150_fst.png added!
202506091200_fst.png added!
202506091210_fst.png added!
202506091220_fst.png added!
202506091230_fst.png added!
202506091240_fst.png added!
202506091250_fst.png added!
202506091300_fst.png added!
/mnt/e/metradar_test/pysteps_nowcast/pic/202506091300_fst.png.gif saved successfully!
